In [20]:
import ftm
import hitdifferentparts
import numpy as np
import matplotlib.pyplot as plt
import torch
from kymatio import Scattering1D
import IPython.display as ipd

In [2]:
def interpolate_sounds_imp(x_coord,y_coord,num,prec,J,w,tau,p,D,alpha):
		"""
		x_coord, y_coord is the point at which you want to interpolate the sounds
		num is number of interpolation points
		prec is distance between interpolated to interpolation points
		"""
		scs = []
		sounds = []
		if num == 2:
			inter = [(x_coord+prec,y_coord),(x_coord-prec,y_coord)]
		elif num == 4:
			inter = [(x_coord+prec,y_coord),(x_coord-prec,y_coord),(x_coord,y_coord+prec),(x_coord,y_coord-prec)]
		for (x,y) in inter:
			y = hitdifferentparts.hitdiffparts(x,y,w,tau,p,D,alpha)
			Sy = getsc(torch.Tensor(y),J)
			sounds.append(y)
			scs.append(Sy)
		return sum(scs)/float(num),sounds

def getsc(y, J):
	"""
	this function outputs scattering transform of a time-domain signal.
	"""
	N = 2**15
	scattering = Scattering1D(J=J, shape=(N,), Q=1, max_order=2)
	Sy = scattering(y)
	return Sy

def interpolate_sounds(x_coord,y_coord,num,prec,J,w,tau,p,D,alpha):
		"""
		x_coord, y_coord is the point at which you want to interpolate the sounds
		num is number of interpolation points
		prec is distance between interpolated to interpolation points
		"""
		scs = []
		sounds = []
		if num == 2:
			inter = [(x_coord+prec,y_coord),(x_coord-prec,y_coord)]
		elif num == 4:
			inter = [(x_coord+prec,y_coord),(x_coord-prec,y_coord),(x_coord,y_coord+prec),(x_coord,y_coord-prec)]
		for (x,y) in inter:
			y = hitdifferentparts.getsounds_imp_gaus(10,10,x,y,w,tau,p,D,alpha,22050)
			Sy = getsc(torch.Tensor(y),J)
			sounds.append(y)
			scs.append(Sy)
		return sum(scs)/float(num),sounds

In [11]:
j = 8
order = 2
eps = 1e-3
w,t,pp,d,a = [3960.1349876368563*2, 0.09376144165196816, 0.0016713390753826927*10,
       0.0012821611585344935, 0.23768976773282824]

In [12]:
#interpolation from scattering
Sy_interpolated,sounds = interpolate_sounds(0.5,0.5,4,0.05,j,w,t,pp,d,a)
Sy_interpolated_log = np.log1p(((Sy_interpolated>0)*Sy_interpolated)/eps)

In [13]:
#accurate scattering
y_acc2 = ftm.getsounds_imp(10,10,w,t,pp,d,a,sr=22050)[:2**15]
y_acc2 = y_acc2/ max(y_acc2)
Sy_acc2 = getsc(torch.Tensor(y_acc2),j)
Sy_acc_log2 = np.log1p(((Sy_acc2>0)*Sy_acc2)/eps)
#interpolation from raw audio
avg_sound = np.sum(sounds,axis=0)/4
Sy_raw = getsc(torch.Tensor(avg_sound),j)
Sy_raw = np.log1p(((Sy_raw>0)*Sy_raw)/eps)

In [14]:
print(np.linalg.norm(Sy_acc_log2-Sy_raw),np.linalg.norm(Sy_acc_log2-Sy_interpolated_log),np.linalg.norm(Sy_raw),
     np.linalg.norm(Sy_acc_log2),np.linalg.norm(Sy_interpolated_log))

0.6500606 0.62157077 37.37163 37.38684 37.38101


this shows that interpolating with scattering yields slightly more accurate result than with raw audio, but not much??

In [17]:
b = [1,2,3]
b_1 = np.zeros(4)
b_1[1:] = b[:]
num = np.random.randint(1,10)
print(b,b_1,num)

[1, 2, 3] [0. 1. 2. 3.] 5


In [45]:
sound_longer = np.zeros(2**16)
Sy_sum = []
for sound in sounds:
    num = np.random.randint(1,10)
    sound_blank = np.zeros(2**16)
    sound_blank[num:num+2**15] = sound[:]
    Sy_sum.append(getsc(torch.Tensor(sound_blank[:2**15]),j))
    sound_longer[num:num+2**15] += sound[:]
    print(num)
Sy_inter = sum(Sy_sum)/4
Sy_inter = np.log1p(((Sy_inter>0)*Sy_inter)/eps)
sound_longer = sound_longer[:2**15]/4
Sy_raw = getsc(torch.Tensor(sound_longer),j)
Sy_raw = np.log1p(((Sy_raw>0)*Sy_raw)/eps)

6
2
3
6


In [46]:
ipd.Audio(sound_longer,rate=22050)

In [47]:
print(np.linalg.norm(Sy_acc_log2-Sy_raw),np.linalg.norm(Sy_acc_log2-Sy_interpolated_log),
      np.linalg.norm(Sy_acc_log2-Sy_inter),np.linalg.norm(Sy_raw),
     np.linalg.norm(Sy_acc_log2),np.linalg.norm(Sy_interpolated_log))

3.5481102 0.62157077 0.63930804 34.715195 37.38684 37.38101
